In [90]:
import kuzu

# Create an empty on-disk database and connect to it
db = kuzu.Database("./demo_db")
conn = kuzu.Connection(db)
# Create a graph with a node label and a property
try:
    conn.execute(
        """
          CREATE NODE TABLE Movie (name STRING, PRIMARY KEY(name));
          CREATE NODE TABLE Person (name STRING, birthDate STRING, PRIMARY KEY(name));
          CREATE REL TABLE ActedIn (FROM Person TO Movie);
          CREATE (:Person {name: 'Al Pacino', birthDate: '1940-04-25'});
          CREATE (:Person {name: 'Robert De Nero', birthDate: '1943-08-17'});
          CREATE (:Movie {name: 'The Godfather: Part II'});
          MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m);
          MATCH (p:Person), (m:Movie) WHERE p.name = 'Robert De Nero' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m);
        """
    )
except Exception as e:
    print(f"Error executing graph operations: {str(e)}")
# Execute Cypher query
response = conn.execute(
    """
    MATCH (a)
    RETURN a.name AS name 
    """
)
while response.has_next():
    print(response.get_next())
response.close()
conn.close()
db.close()

Error executing graph operations: Binder exception: Movie already exists in catalog.
['Al Pacino']
['Robert De Nero']
['The Godfather: Part II']


In [13]:
import kuzu


class KuzuSession:
    def __init__(self, db_path):
        self.db_path = db_path

    def __enter__(self):
        self.db = kuzu.Database(self.db_path)
        self.conn = kuzu.Connection(self.db)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
       # self.conn.close()
       # self.db.close()
        print("Session closed successfully.")

# Usage:
with KuzuSession("demo_db") as conn:
    result = conn.execute("MATCH (n) RETURN n LIMIT 1")
    print(result.get_as_df())

RuntimeError: IO exception: Could not set lock on file : e:\projects\kuzu-lite\util\demo_db/.lock
See the docs: https://docs.kuzudb.com/concurrency for more information.

In [3]:
from safe_kuzu_subprocess import SafeKuzuSubprocess
with SafeKuzuSubprocess("demo_db") as db:
    df = db.execute("MATCH (n)-[r]-(m) RETURN * LIMIT 10")
    print(df)

                                                   n  \
0  {'_id': {'offset': 0, 'table': 0}, '_label': '...   
1  {'_id': {'offset': 0, 'table': 0}, '_label': '...   
2  {'_id': {'offset': 1, 'table': 1}, '_label': '...   
3  {'_id': {'offset': 0, 'table': 1}, '_label': '...   

                                                   m  \
0  {'_id': {'offset': 0, 'table': 1}, '_label': '...   
1  {'_id': {'offset': 1, 'table': 1}, '_label': '...   
2  {'_id': {'offset': 0, 'table': 0}, '_label': '...   
3  {'_id': {'offset': 0, 'table': 0}, '_label': '...   

                                                   r  
0  {'_src': {'offset': 0, 'table': 1}, '_dst': {'...  
1  {'_src': {'offset': 1, 'table': 1}, '_dst': {'...  
2  {'_src': {'offset': 1, 'table': 1}, '_dst': {'...  
3  {'_src': {'offset': 0, 'table': 1}, '_dst': {'...  


In [4]:

from safe_kuzu_subprocess import SafeKuzuSubprocess
with SafeKuzuSubprocess("demo_large") as db:
    df = db.execute("MATCH (n)-[r]-(m) RETURN * LIMIT 2000")
    print(df)

                                                      n  \
0     {'_id': {'offset': 70014, 'table': 1}, '_label...   
1     {'_id': {'offset': 70015, 'table': 1}, '_label...   
2     {'_id': {'offset': 70016, 'table': 1}, '_label...   
3     {'_id': {'offset': 70017, 'table': 1}, '_label...   
4     {'_id': {'offset': 70018, 'table': 1}, '_label...   
...                                                 ...   
1995  {'_id': {'offset': 183, 'table': 0}, '_label':...   
1996  {'_id': {'offset': 183, 'table': 0}, '_label':...   
1997  {'_id': {'offset': 183, 'table': 0}, '_label':...   
1998  {'_id': {'offset': 183, 'table': 0}, '_label':...   
1999  {'_id': {'offset': 183, 'table': 0}, '_label':...   

                                                      m  \
0     {'_id': {'offset': 0, 'table': 0}, '_label': '...   
1     {'_id': {'offset': 0, 'table': 0}, '_label': '...   
2     {'_id': {'offset': 0, 'table': 0}, '_label': '...   
3     {'_id': {'offset': 0, 'table': 0}, '_label': '...